In [31]:
import os
import psycopg2
from psycopg2.extras import RealDictCursor
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding

In [32]:
# config.py
DB_CONFIG = {
    "dbname": "enc2db_lab",
    "user": "postgres",
    "password": "Password123",      # nếu Postgres có password thì điền vào
    "host": "192.168.99.9",
    "port": 5432,
}
# 32 bytes = 256-bit AES key (DEMO: hard-code, thực tế phải lưu an toàn)
AES_KEY = b"0123456789abcdef0123456789abcdef"

# ORE (toy) cho numeric: c = A * x_scaled + B
# A > 0 để bảo toàn thứ tự. A,B là bí mật, chỉ client biết.

ORE_A = 7919          # prime > 0
ORE_B = 123456789123  # offset bí mật
ORE_SCALE = 100       # scale cho 2 chữ số thập phân (18,2)

# AHE numeric (toy): c = x_scaled + AHE_K
AHE_K = 987654321  # secret offset
AHE_SCALE = 100    # dùng chung scale 2 decimals cho dễ

# MHE numeric (toy): c = x_scaled * MHE_K
MHE_K = 123457        # prime-ish secret
MHE_SCALE = 5       # dùng chung 2 decimals cho dễ

## Step 2: ENCDB (Sofware Only)
### Step 2.1: Encryption and Decryption Engine (on Client)
Using EAS Encryption Algorithm

In [33]:
# aes_engine.py
def aes_encrypt(plaintext: str) -> bytes:
    """
    Mã hóa AES-256-CBC với PKCS7 padding.
    Lưu trên DB dạng: iv || ciphertext
    """
    if plaintext is None:
        raise ValueError("plaintext is None")

    iv = os.urandom(16)  # 128-bit IV
    cipher = Cipher(algorithms.AES(AES_KEY), modes.CBC(iv))
    encryptor = cipher.encryptor()

    padder = padding.PKCS7(algorithms.AES.block_size).padder()
    padded_data = padder.update(plaintext.encode("utf-8")) + padder.finalize()

    ciphertext = encryptor.update(padded_data) + encryptor.finalize()
    return iv + ciphertext  # gộp iv + ciphertext để lưu vào 1 cột BYTEA

def aes_decrypt(cipher_blob: bytes) -> str:
    """
    Giải mã AES-256-CBC với PKCS7 padding.
    Đầu vào: iv || ciphertext (như aes_encrypt xuất ra)
    """
    if cipher_blob is None:
        raise ValueError("cipher_blob is None")

    iv = cipher_blob[:16]
    ciphertext = cipher_blob[16:]

    cipher = Cipher(algorithms.AES(AES_KEY), modes.CBC(iv))
    decryptor = cipher.decryptor()
    padded_plain = decryptor.update(ciphertext) + decryptor.finalize()

    unpadder = padding.PKCS7(algorithms.AES.block_size).unpadder()
    plaintext_bytes = unpadder.update(padded_plain) + unpadder.finalize()

    return plaintext_bytes.decode("utf-8")


In [34]:
# aes_engine.py
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
# from config import AES_KEY


FIXED_IV = b"\x00" * 16   # 👈 IV cố định cho demo equality


def aes_encrypt2(plaintext: str) -> bytes:
    if plaintext is None:
        raise ValueError("plaintext is None")

    cipher = Cipher(algorithms.AES(AES_KEY), modes.CBC(FIXED_IV))
    encryptor = cipher.encryptor()

    padder = padding.PKCS7(algorithms.AES.block_size).padder()
    padded = padder.update(plaintext.encode("utf-8")) + padder.finalize()

    ct = encryptor.update(padded) + encryptor.finalize()
    return ct   # 👈 không cần prepend IV nữa, vì IV cố định


def aes_decrypt2(blob: bytes) -> str:
    if blob is None:
        raise ValueError("cipher blob is None")

    cipher = Cipher(algorithms.AES(AES_KEY), modes.CBC(FIXED_IV))
    decryptor = cipher.decryptor()
    padded = decryptor.update(blob) + decryptor.finalize()

    unpadder = padding.PKCS7(algorithms.AES.block_size).unpadder()
    plain_bytes = unpadder.update(padded) + unpadder.finalize()
    return plain_bytes.decode("utf-8")


In [35]:
# ahe_numeric.py
from decimal import Decimal
def ahe_encrypt_numeric(x: float | Decimal) -> int:
    """
    Toy AHE:
      x_scaled = round(x * SCALE)
      c = x_scaled + AHE_K
    Server chỉ thấy c, không biết x.
    """
    if x is None:
        raise ValueError("ahe_encrypt_numeric: value is None")

    dx = Decimal(str(x))
    scaled = int(dx * AHE_SCALE)
    return scaled + AHE_K


def ahe_decrypt_single(c: int) -> float:
    """
    Giải mã 1 ciphertext đơn lẻ, chủ yếu để debug.
    """
    scaled = c - AHE_K
    return scaled / AHE_SCALE


def ahe_decrypt_sum(sum_cipher: int, n: int) -> float:
    """
    sum_cipher = sum(x_scaled + AHE_K) = sum(x_scaled) + n*AHE_K
    => sum(x) = (sum_cipher - n*AHE_K) / SCALE
    """
    sum_scaled = sum_cipher - n * AHE_K
    return sum_scaled / AHE_SCALE

In [36]:
# mhe_numeric.py
from decimal import Decimal
# from config import MHE_K, MHE_SCALE


def mhe_encrypt_numeric(x: float | Decimal) -> int:
    """
    Toy MHE:
      x_scaled = round(x * SCALE)
      c = x_scaled * MHE_K
    Server chỉ nhân ciphertext.
    """
    if x is None:
        raise ValueError("mhe_encrypt_numeric: value is None")

    dx = Decimal(str(x))
    scaled = int(dx * MHE_SCALE)
    return scaled * MHE_K


def mhe_decrypt_product(prod_cipher: int, n: int) -> float:
    """
    Với n phần tử:
      prod_cipher = Π (xi_scaled * MHE_K)
                  = (Π xi_scaled) * (MHE_K ** n)

      => Π xi = (Π xi_scaled) / (SCALE ** n)
              = (prod_cipher / (MHE_K ** n)) / (MHE_SCALE ** n)

    Ở đây ta trả về "product" theo đúng scale (cẩn thận overflow và độ lớn).
    Mục đích chính là minh họa homomorphic, không phải để dùng thực tế.
    """
    from decimal import Decimal, getcontext

    # tăng precision để tránh mất mát khi n lớn
    getcontext().prec = 50

    prod_dec = Decimal(prod_cipher)
    k_pow_n = Decimal(MHE_K) ** n
    scale_pow_n = Decimal(MHE_SCALE) ** n

    prod_scaled = prod_dec / k_pow_n
    prod_plain = prod_scaled / scale_pow_n

    return float(prod_plain)


In [38]:
# ore.py
from decimal import Decimal
def ore_encrypt_numeric(x: float | Decimal) -> int:
    """
    ORE cho numeric (18,2):
      - scale x lên integer: x_scaled = round(x * SCALE)
      - ciphertext: c = A * x_scaled + B
    Bởi vì A > 0, nên:
      x1 < x2  =>  x1_scaled < x2_scaled  =>  c1 < c2
    """
    if x is None:
        raise ValueError("ore_encrypt_numeric: value is None")

    # làm việc với Decimal cho ổn hơn
    dx = Decimal(str(x))
    scaled = int(dx * ORE_SCALE)

    c = ORE_A * scaled + ORE_B
    return c

In [27]:
def get_conn():
    return psycopg2.connect(**DB_CONFIG)

In [39]:
# ---------- INSERT QUERY REWRITE ----------
def insert_user(username: str, email: str, phone: str):
    """
    Rewrite INSERT:
      - plaintext email, phone -> aes_encrypt -> email_cipher, phone_cipher
      - gửi INSERT xuống Postgres
    """
    email_cipher = aes_encrypt(email)
    phone_cipher = aes_encrypt(phone)

    sql = """
        INSERT INTO tbl_encrypt_aes (username, email_cipher, phone_cipher)
        VALUES (%s, %s, %s)
    """

    with get_conn() as conn:
        with conn.cursor() as cur:
            cur.execute(sql, (username, psycopg2.Binary(email_cipher),
                              psycopg2.Binary(phone_cipher)))
        conn.commit()

# ---------- SELECT QUERY REWRITE ----------
def select_all_users():
    """
    Rewrite SELECT:
      - SELECT id, username, email_cipher, phone_cipher
      - dùng aes_decrypt để trả về email, phone dạng plaintext
    """
    sql = """
        SELECT id, username, email_cipher, phone_cipher
        FROM tbl_encrypt_aes
        ORDER BY id;
    """

    with get_conn() as conn:
        with conn.cursor(cursor_factory=RealDictCursor) as cur:
            cur.execute(sql)
            rows = cur.fetchall()

    result = []
    for r in rows:
        result.append({
            "id": r["id"],
            "username": r["username"],
            "email": aes_decrypt(r["email_cipher"]),
            "phone": aes_decrypt(r["phone_cipher"]),
        })
    return result


In [40]:
def insert_numeric(plain_value: float):
    """
    Ghi 1 dòng vào tbl_enc_numeric:
      - plain_value: lưu để debug / demo
      - ore_cipher: ORE ciphertext
      - ahe_cipher, mhe_cipher: tạm bỏ trống
    """
    ore_ct = ore_encrypt_numeric(plain_value)

    sql = """
        INSERT INTO tbl_enc_numeric (plain_value, ahe_cipher, mhe_cipher, ore_cipher)
        VALUES (%s, NULL, NULL, %s)
    """

    with get_conn() as conn:
        with conn.cursor() as cur:
            cur.execute(sql, (Decimal(str(plain_value)), ore_ct))
        conn.commit()

def insert_numeric2(plain_value: float):
    from decimal import Decimal
    

    ore_ct = ore_encrypt_numeric(plain_value)

    sql = """
        INSERT INTO tbl_enc_numeric2 (plain_value, ore_cipher)
        VALUES (%s, %s::ore_en)
    """

    with psycopg2.connect(**DB_CONFIG) as conn:
        with conn.cursor() as cur:
            cur.execute(sql, (Decimal(str(plain_value)), ore_ct))
        conn.commit()

def query_greater_than(threshold: float):
    """
    Demo query:
    SELECT * FROM tbl_enc_numeric
    WHERE udf_ore_gt(ore_cipher, enc(threshold))
    ORDER BY ore_cipher;
    """
    ct_threshold = ore_encrypt_numeric(threshold)

    sql = """
        SELECT id, plain_value, ore_cipher
        FROM tbl_enc_numeric
        WHERE udf_ore_gt(ore_cipher, %s)
        ORDER BY ore_cipher;
    """

    with get_conn() as conn:
        with conn.cursor(cursor_factory=RealDictCursor) as cur:
            cur.execute(sql, (ct_threshold,))
            rows = cur.fetchall()

    return rows

def query_greater_than2(threshold: float):
    """
    Demo query:
    SELECT id, plain_value
        FROM tbl_enc_numeric2
        WHERE ore_cipher > :enc_20
        ORDER BY ore_cipher;
    """
    ct_threshold = ore_encrypt_numeric(threshold)

    sql = """
        SELECT id, plain_value
        FROM tbl_enc_numeric2
        WHERE ore_cipher > %s
        ORDER BY ore_cipher;
    """

    with get_conn() as conn:
        with conn.cursor(cursor_factory=RealDictCursor) as cur:
            cur.execute(sql, (ct_threshold,))
            rows = cur.fetchall()

    return rows

In [41]:
def insert_row_enc_all(plain_text: str, plain_numeric: float):
    aes_ct = aes_encrypt2(plain_text)
    ore_ct = ore_encrypt_numeric(plain_numeric)
    ahe_ct = ahe_encrypt_numeric(plain_numeric)
    mhe_ct = mhe_encrypt_numeric(plain_numeric)

    sql = """
        INSERT INTO tbl_enc_all
            (plain_text, plain_numeric,
             aes_cipher, ore_cipher, ahe_cipher, mhe_cipher)
        VALUES (%s, %s, %s, %s::ore_en, %s, %s)
    """

    with get_conn() as conn:
        with conn.cursor() as cur:
            cur.execute(
                sql,
                (
                    plain_text,
                    Decimal(str(plain_numeric)),
                    psycopg2.Binary(aes_ct),
                    ore_ct,
                    ahe_ct,
                    mhe_ct,
                ),
            )
        conn.commit()

In [42]:
def query_by_aes_eq(plain_text: str):
    aes_ct = aes_encrypt2(plain_text)

    sql = """
        SELECT id, aes_cipher, plain_text, plain_numeric
        FROM tbl_enc_all
        WHERE udf_aes_eq(aes_cipher, %s::bytea)
    """

    with get_conn() as conn:
        with conn.cursor(cursor_factory=RealDictCursor) as cur:
            cur.execute(sql, (psycopg2.Binary(aes_ct),))
            return cur.fetchall()


In [43]:
def query_ore_gt(threshold: float):
    ore_ct = ore_encrypt_numeric(threshold)

    sql = """
        SELECT id, plain_text, plain_numeric, ore_cipher
        FROM tbl_enc_all
        WHERE ore_cipher > %s::ore_en
        ORDER BY ore_cipher;
    """

    with get_conn() as conn:
        with conn.cursor(cursor_factory=RealDictCursor) as cur:
            cur.execute(sql, (ore_ct,))
            return cur.fetchall()

In [44]:
def query_ahe_sum():
    """
    Demo AHE:
      - Server: ahe_sum(ahe_cipher), COUNT(*)
      - Client: ahe_decrypt_sum(sum_cipher, n)
    """
    sql = """
        SELECT ahe_sum(ahe_cipher) AS sum_cipher,
               COUNT(*)            AS n
        FROM tbl_enc_all;
    """

    with get_conn() as conn:
        with conn.cursor(cursor_factory=RealDictCursor) as cur:
            cur.execute(sql)
            row = cur.fetchone()

    sum_cipher = row["sum_cipher"]
    n = row["n"]

    sum_plain = ahe_decrypt_sum(sum_cipher, n)
    return sum_cipher, n, sum_plain


In [45]:
def query_mhe_prod():
    sql = """
        SELECT mhe_prod(mhe_cipher) AS prod_cipher,
               COUNT(*)             AS n
        FROM tbl_enc_all;
    """

    with get_conn() as conn:
        with conn.cursor(cursor_factory=RealDictCursor) as cur:
            cur.execute(sql)
            row = cur.fetchone()

    prod_cipher = row["prod_cipher"]
    n = row["n"]

    prod_plain = mhe_decrypt_product(prod_cipher, n)
    return prod_cipher, n, prod_plain


In [46]:
def demo3():
    with get_conn() as conn:
        with conn.cursor() as cur:
            cur.execute("DELETE FROM tbl_enc_all;")
        conn.commit()

    print("== Insert sample rows ==")
    insert_row_enc_all("Alice secret", 10.50)
    insert_row_enc_all("Bob secret",   20.00)
    insert_row_enc_all("Charlie note", 35.75)
    insert_row_enc_all("David note",   50.25)

    print("== Raw rows (for debug) ==")
    with get_conn() as conn:
        with conn.cursor(cursor_factory=RealDictCursor) as cur:
            cur.execute("SELECT * FROM tbl_enc_all ORDER BY id;")
            for r in cur.fetchall():
                print(r)

    print("== Query by AES equal (plain_text='Bob secret') ==")
    for r in query_by_aes_eq("Bob secret"):
        print(r)

    print("== Query numeric > 20.00 via ORE (and index) ==")
    for r in query_ore_gt(20.00):
        print(r)

    print("== AHE sum over plain_numeric ==")
    sum_cipher, n, sum_plain = query_ahe_sum()
    print(f"sum_cipher = {sum_cipher}, n = {n}, decrypted sum = {sum_plain}")

In [47]:
def demo():
    # Xóa dữ liệu cũ cho dễ test
    with get_conn() as conn:
        with conn.cursor() as cur:
            cur.execute("DELETE FROM tbl_encrypt_aes")
            cur.execute("DELETE FROM tbl_enc_numeric")
        conn.commit()

    print("== Insert sample data (plaintext) ==")
    insert_user("Alice", "alice@example.com", "+84123456789")
    insert_user("Bob",   "bob@example.com",   "+84987654321")

    print("== Raw data trong DB (ciphertext) ==")
    with get_conn() as conn:
        with conn.cursor(cursor_factory=RealDictCursor) as cur:
            cur.execute("SELECT * FROM tbl_encrypt_aes ORDER BY id;")
            for row in cur.fetchall():
                print(row)  # email_cipher/phone_cipher là BYTEA

    print("== Select_all_users() (đã decrypt về plaintext) ==")
    for user in select_all_users():
        print(user)

    ## ORE demo
    print("== Insert sample numeric values ==")
    insert_numeric2(10.50)
    insert_numeric2(20.00)
    insert_numeric2(35.75)
    insert_numeric2(50.25)

    print("== Raw data in DB (cipher) ==")
    with get_conn() as conn:
        with conn.cursor(cursor_factory=RealDictCursor) as cur:
            cur.execute("SELECT * FROM tbl_enc_numeric2 ORDER BY id;")
            for row in cur.fetchall():
                print(row)

    print("== Query: plain_value > 20.00 (via ORE) ==")
    res = query_greater_than2(20.00)
    for r in res:
        print(r)

In [48]:
def demo2():
    print("== Raw data in DB (cipher) ==")
    with get_conn() as conn:
        with conn.cursor(cursor_factory=RealDictCursor) as cur:
            cur.execute("SELECT * FROM tbl_enc_numeric2 ORDER BY id;")
            for row in cur.fetchall():
                print(row)
    print("== Query: plain_value > 20.00 (via ORE) ==")
    res = query_greater_than2(20.00)
    for r in res:
        print(r)

In [49]:
def demo4():
    with get_conn() as conn:
        with conn.cursor() as cur:
            cur.execute("DELETE FROM tbl_enc_all;")
        conn.commit()

    print("== Insert sample rows ==")
    insert_row_enc_all("Alice secret", 2.0)
    insert_row_enc_all("Bob secret",   3.0)
    insert_row_enc_all("Charlie note", 4.0)

    print("== Raw rows (for debug) ==")
    with get_conn() as conn:
        with conn.cursor(cursor_factory=RealDictCursor) as cur:
            cur.execute("SELECT * FROM tbl_enc_all ORDER BY id;")
            for r in cur.fetchall():
                print(r)

    print("== Query by AES equal (plain_text='Bob secret') ==")
    for r in query_by_aes_eq("Bob secret"):
        print(r)

    print("== Query numeric > 2.0 via ORE (and index) ==")
    for r in query_ore_gt(2.0):
        print(r)

    print("== AHE sum over plain_numeric ==")
    sum_cipher, n_sum, sum_plain = query_ahe_sum()
    print(f"AHE: sum_cipher = {sum_cipher}, n = {n_sum}, decrypted sum = {sum_plain}")

    print("== MHE product over plain_numeric ==")
    prod_cipher, n_prod, prod_plain = query_mhe_prod()
    print(f"MHE: prod_cipher = {prod_cipher}, n = {n_prod}, decrypted product = {prod_plain}")




In [51]:
demo()

== Insert sample data (plaintext) ==
== Raw data trong DB (ciphertext) ==
RealDictRow({'id': 7, 'username': 'Alice', 'email_cipher': <memory at 0x000002336C7C3E80>, 'phone_cipher': <memory at 0x000002336CF58400>})
RealDictRow({'id': 8, 'username': 'Bob', 'email_cipher': <memory at 0x000002336CF59540>, 'phone_cipher': <memory at 0x000002336CF584C0>})
== Select_all_users() (đã decrypt về plaintext) ==
{'id': 7, 'username': 'Alice', 'email': 'alice@example.com', 'phone': '+84123456789'}
{'id': 8, 'username': 'Bob', 'email': 'bob@example.com', 'phone': '+84987654321'}
== Insert sample numeric values ==
== Raw data in DB (cipher) ==
RealDictRow({'id': 1, 'plain_value': Decimal('10.50'), 'ore_cipher': 123465104073})
RealDictRow({'id': 2, 'plain_value': Decimal('20.00'), 'ore_cipher': 123472627123})
RealDictRow({'id': 3, 'plain_value': Decimal('35.75'), 'ore_cipher': 123485099548})
RealDictRow({'id': 4, 'plain_value': Decimal('50.25'), 'ore_cipher': 123496582098})
RealDictRow({'id': 5, 'plain

In [50]:
# demo()
#demo2()
demo4()

== Insert sample rows ==
== Raw rows (for debug) ==
RealDictRow({'id': 7, 'plain_text': 'Alice secret', 'plain_numeric': Decimal('2.00'), 'aes_cipher': <memory at 0x000002336C7C3E80>, 'ore_cipher': 123458372923, 'ahe_cipher': 987654521, 'mhe_cipher': 1234570})
RealDictRow({'id': 8, 'plain_text': 'Bob secret', 'plain_numeric': Decimal('3.00'), 'aes_cipher': <memory at 0x000002336CF584C0>, 'ore_cipher': 123459164823, 'ahe_cipher': 987654621, 'mhe_cipher': 1851855})
RealDictRow({'id': 9, 'plain_text': 'Charlie note', 'plain_numeric': Decimal('4.00'), 'aes_cipher': <memory at 0x000002336CF59540>, 'ore_cipher': 123459956723, 'ahe_cipher': 987654721, 'mhe_cipher': 2469140})
== Query by AES equal (plain_text='Bob secret') ==
RealDictRow({'id': 8, 'aes_cipher': <memory at 0x000002336C7C3E80>, 'plain_text': 'Bob secret', 'plain_numeric': Decimal('3.00')})
== Query numeric > 2.0 via ORE (and index) ==
RealDictRow({'id': 8, 'plain_text': 'Bob secret', 'plain_numeric': Decimal('3.00'), 'ore_cipher